In [1]:
try:
    import openmdao.api as om
except ImportError:
    !python -m pip install openmdao[notebooks]
    import openmdao.api as om

# Building Models with Solvers and Implicit Components
---
This tutorial will show you how to define implicit components and build models with them.
We'll use a nonlinear circuit analysis example problem.

## Circuit analysis
---
Consider a simple electrical circuit made up from two resistors, a diode, and a constant current source.
Our goal is to solve for the steady-state voltages at node 1 and node 2.

![circuit_diagram](images/circuit_diagram.png)

In order to find the voltages, we'll employ [Kirchoff's current law](https://en.wikipedia.org/wiki/Kirchhoff%27s_circuit_laws),
and solve for the voltages needed at each node to drive the net current to 0.

This means that the voltages at each node are *state variables* for the analysis.
In other words, V1 and V2 are defined implicitly by the following residual equation:

${R_{node_j}} = \sum_k I_{k}^{in} - \sum_k I_{k}^{out} = 0 .$

To build this model we're going to define three different components:

1. Resistor (Explicit)
2. Diode (Explicit)
3. Node (Implicit)

ExplicitComponents - Resistor and Diode
---

The `Resistor` and `Diode` components will each compute their current, given the voltages on either side.
These calculations are analytic functions, so we'll inherit from :ref:`ExplicitComponent <comp-type-2-explicitcomp>`.
These components will each declare some options to allow you to pass in the relevant physical constants, and to
allow you to give some reasonable default values.

In [ ]:
class Resistor(om.ExplicitComponent):
    """Computes current across a resistor using Ohm's law."""

    def initialize(self):
        self.options.declare('R', default=1., desc='Resistance in Ohms')

    def setup(self):
        self.add_input('V_in', units='V')
        self.add_input('V_out', units='V')
        self.add_output('I', units='A')

    def setup_partials(self):
        self.declare_partials('I', 'V_in', method='fd')
        self.declare_partials('I', 'V_out', method='fd')

    def compute(self, inputs, outputs):
        deltaV = inputs['V_in'] - inputs['V_out']
        outputs['I'] = deltaV / self.options['R']

In [ ]:
class Diode(om.ExplicitComponent):
    """Computes current across a diode using the Shockley diode equation."""

    def initialize(self):
        self.options.declare('Is', default=1e-15, desc='Saturation current in Amps')
        self.options.declare('Vt', default=.025875, desc='Thermal voltage in Volts')

    def setup(self):
        self.add_input('V_in', units='V')
        self.add_input('V_out', units='V')
        self.add_output('I', units='A')

    def setup_partials(self):
        self.declare_partials('I', 'V_in', method='fd')
        self.declare_partials('I', 'V_out', method='fd')

    def compute(self, inputs, outputs):
        deltaV = inputs['V_in'] - inputs['V_out']
        Is = self.options['Is']
        Vt = self.options['Vt']
        outputs['I'] = Is * (np.exp(deltaV / Vt) - 1)

```{important}
Since we've provided default values for the options, they won't be required arguments when instantiating `Resistor` or `Diode`. Check out the :ref:`Features <Features>` section for more details on how to use :ref:`component options <component_options>`.
```

## ImplicitComponent - Node
---

The `Node` component inherits from :ref:`ImplicitComponent <comp-type-3-implicitcomp>`, which has a different interface than :ref:`ExplicitComponent <comp-type-2-explicitcomp>`.
Rather than compute the values of its outputs, it computes residuals via the `apply_nonlinear` method.
When those residuals have been driven to zero, the values of the outputs will be implicitly known.
`apply_nonlinear` computes the `residuals` using values from `inputs` and `outputs`.
Notice that we still define *V* as an output of the `Node` component, albeit one that is implicitly defined.

In [ ]:
class Node(om.ImplicitComponent):
    """Computes voltage residual across a node based on incoming and outgoing current."""

    def initialize(self):
        self.options.declare('n_in', default=1, types=int, desc='number of connections with + assumed in')
        self.options.declare('n_out', default=1, types=int, desc='number of current connections + assumed out')

    def setup(self):
        self.add_output('V', val=5., units='V')

        for i in range(self.options['n_in']):
            i_name = 'I_in:{}'.format(i)
            self.add_input(i_name, units='A')

        for i in range(self.options['n_out']):
            i_name = 'I_out:{}'.format(i)
            self.add_input(i_name, units='A')

    def setup_partials(self):
        #note: we don't declare any partials wrt `V` here,
        #      because the residual doesn't directly depend on it
        self.declare_partials('V', 'I*', method='fd')

    def apply_nonlinear(self, inputs, outputs, residuals):
        residuals['V'] = 0.
        for i_conn in range(self.options['n_in']):
            residuals['V'] += inputs['I_in:{}'.format(i_conn)]
        for i_conn in range(self.options['n_out']):
            residuals['V'] -= inputs['I_out:{}'.format(i_conn)]

Every state variable must have exactly one corresponding residual which is defined in the `apply_nonlinear` method. The `residuals` equations in an implicit component are not analogous to the `outputs` equations in the `compute` method of an explicit component. Instead of defining an explicit equation for the output, `residuals['example_output']` defines an equation for the residual *associated with* the output (state variable) `example_output`. In our example, `residuals['V']` defines the equation of the residual associated with the state variable *V*. There will be no explicit
equation defining *V*, instead, the residual equation sums the currents associated with *V* so the sum can be driven to zero.

An implicit component varies its outputs (state variables, in this case *V*) to drive the residual equation to zero. In our model, *V* does not show up directly in the residual equation. Instead, our explicit components `Resistor` and `Diode` create a dependence of the currents on *V*, so by using a solver on a higher level of the model hierarchy, we can vary *V* to have an effect on current, and we can drive the residuals to zero.

All implicit components must define the `apply_nonlinear` method, but it is not a requirement that every :ref:`ImplicitComponent <comp-type-3-implicitcomp>`  define the `solve_nonlinear` method. (The `solve_nonlinear` method provides a way to explicitly define an output within an implicit component.) In fact, for the `Node` component, it is not even possible to define a `solve_nonlinear` because *V* does not show up directly in the residual function. So the implicit function represented by instances of the `Node` component must be converged at a higher level in the model hierarchy.

There are cases where it is possible, and even advantageous, to define the `solve_nonlinear` method.
For example, when a component is performing an engineering analysis with its own specialized nonlinear solver routines (e.g. CFD or FEM), then it makes sense to expose those to OpenMDAO via `solve_nonlinear` so OpenMDAO can make use of them. Just remember that `apply_nonlinear` must be defined, regardless of whether you also define `solve_nonlinear`.

```{important}
In this case, the residual equation is not a direct function of the state variable *V*. Often, however, the residual might be a direct function of one or more output variables. If that is the case, you can access the values via `outputs['V']`. See the :ref:`ImplicitComponent <comp-type-3-implicitcomp>` documentation for an example of this.
```

## Building the Circuit Group and Solving It with NewtonSolver
---
We can combine the `Resistor`, `Diode`, and `Node` into the circuit pictured above using a :ref:`Group <feature_grouping_components>`. Adding components and connecting their variables is the same as what you've seen before in the :ref:`Sellar - Two Discipline <sellar>` tutorial. What is new here is the additional use of the nonlinear :ref:`NewtonSolver <nlnewton>` and linear :ref:`DirectSolver <directsolver>` to converge the system.

In previous tutorials, we used a gradient-free :ref:`NonlinearBlockGaussSeidel <nlbgs>` solver, but that won't work here. Just above, we discussed that the `Node` class does not, and in fact can not, define its own `solve_nonlinear` method. Hence, there would be no calculations for the GaussSeidel solver to iterate on.
Instead we use the Newton solver at the `Circuit` level, which uses Jacobian information to compute group level updates for all the variables simultaneously. The Newton solver's use of that Jacobian information is why we need to declare a linear solver in this case.

```{important}
OpenMDAO provides a library of :ref:`linear solvers <feature_linear_solvers>` that are useful in different advanced scenarios. For many problems, especially problems built from components with mostly scalar variables, the :ref:`DirectSolver <directsolver>` will be both the most efficient and the easiest to use. We recommend you stick with :ref:`DirectSolver <directsolver>` unless you have a good reason to switch.
```

In [ ]:
from openmdao.test_suite.scripts.circuit_analysis import Resistor, Diode, Node

class Circuit(om.Group):

    def setup(self):
        self.add_subsystem('n1', Node(n_in=1, n_out=2), promotes_inputs=[('I_in:0', 'I_in')])
        self.add_subsystem('n2', Node())  # leaving defaults

        self.add_subsystem('R1', Resistor(R=100.), promotes_inputs=[('V_out', 'Vg')])
        self.add_subsystem('R2', Resistor(R=10000.))
        self.add_subsystem('D1', Diode(), promotes_inputs=[('V_out', 'Vg')])

        self.connect('n1.V', ['R1.V_in', 'R2.V_in'])
        self.connect('R1.I', 'n1.I_out:0')
        self.connect('R2.I', 'n1.I_out:1')

        self.connect('n2.V', ['R2.V_out', 'D1.V_in'])
        self.connect('R2.I', 'n2.I_in:0')
        self.connect('D1.I', 'n2.I_out:0')

        self.nonlinear_solver = om.NewtonSolver(solve_subsystems=False)
        self.nonlinear_solver.options['iprint'] = 2
        self.nonlinear_solver.options['maxiter'] = 20
        self.linear_solver = om.DirectSolver()

p = om.Problem()
model = p.model

model.add_subsystem('ground', om.IndepVarComp('V', 0., units='V'))
model.add_subsystem('source', om.IndepVarComp('I', 0.1, units='A'))
model.add_subsystem('circuit', Circuit())

model.connect('source.I', 'circuit.I_in')
model.connect('ground.V', 'circuit.Vg')

p.setup()

# set some initial guesses
p['circuit.n1.V'] = 10.
p['circuit.n2.V'] = 1.

p.run_model()

print(p['circuit.n1.V'])
print(p['circuit.n2.V'])
print(p['circuit.R1.I'])
print(p['circuit.R2.I'])
print(p['circuit.D1.I'])

# sanity check: should sum to .1 Amps
print(p['circuit.R1.I'] + p['circuit.D1.I'])

In [5]:
from openmdao.utils.assert_utils import assert_near_equal

assert_near_equal(p['circuit.n1.V'], 9.90804735, 1e-5)
assert_near_equal(p['circuit.n2.V'], 0.71278185, 1e-5)
assert_near_equal(p['circuit.R1.I'], 0.09908047, 1e-5)
assert_near_equal(p['circuit.R2.I'], 0.00091953, 1e-5)
assert_near_equal(p['circuit.D1.I'], 0.00091953, 1e-5)

# sanity check: should sum to .1 Amps
assert_near_equal(p['circuit.R1.I'] + p['circuit.D1.I'], .1, 1e-6)

0.0

## Modifying Solver Settings in Your Run Script
---

In the above run script, we set some initial guess values: `prob['n1.V']=10` and `prob['n2.V']=1`.
If you try to play around with those initial guesses a bit, you will see that convergence is really sensitive to
the initial guess you used for *n2.V*.Below we provide a second run script that uses the same `Circuit` group we defined previously, but which additionally modifies some solver settings and initial guesses. If we set the initial guess for `prob['n2.V']=1e-3`, then the model starts out with a massive residual. It also converges much more slowly, so although we gave it more than twice the number of iterations, it doesn't even get close to a converged answer.

In [6]:
from openmdao.test_suite.scripts.circuit_analysis import Circuit

p = om.Problem()
model = p.model

model.add_subsystem('ground', om.IndepVarComp('V', 0., units='V'))
model.add_subsystem('source', om.IndepVarComp('I', 0.1, units='A'))
model.add_subsystem('circuit', Circuit())

model.connect('source.I', 'circuit.I_in')
model.connect('ground.V', 'circuit.Vg')

p.setup()

# you can change the NewtonSolver settings in circuit after setup is called
newton = p.model.circuit.nonlinear_solver
newton.options['maxiter'] = 50

# set some initial guesses
p['circuit.n1.V'] = 10.
p['circuit.n2.V'] = 1e-3

p.run_model()

print(p['circuit.n1.V'])
print(p['circuit.n2.V'])

# sanity check: should sum to .1 Amps
print(p['circuit.R1.I'] + p['circuit.D1.I'])


circuit
NL: Newton 0 ; 2.53337743 1
NL: Newton 1 ; 6.97216645e+152 2.75212306e+152
NL: Newton 2 ; 2.56496626e+152 1.01246906e+152
NL: Newton 3 ; 9.43616587e+151 3.72473748e+151
NL: Newton 4 ; 3.47143851e+151 1.37028082e+151
NL: Newton 5 ; 1.27709554e+151 5.04107884e+150
NL: Newton 6 ; 4.69826271e+150 1.8545451e+150
NL: Newton 7 ; 1.72842766e+150 6.822622e+149
NL: Newton 8 ; 6.35865288e+149 2.50995087e+149
NL: Newton 9 ; 2.33926287e+149 9.23377165e+148
NL: Newton 10 ; 8.60583345e+148 3.39698039e+148
NL: Newton 11 ; 3.16597037e+148 1.24970339e+148
NL: Newton 12 ; 1.16471792e+148 4.59749069e+147
NL: Newton 13 ; 4.28484056e+147 1.69135499e+147
NL: Newton 14 ; 1.57633521e+147 6.22226752e+146
NL: Newton 15 ; 5.79912522e+146 2.28908853e+146
NL: Newton 16 ; 2.13342017e+146 8.42124882e+145
NL: Newton 17 ; 7.84856585e+145 3.09806417e+145
NL: Newton 18 ; 2.88738181e+145 1.13973614e+145
NL: Newton 19 ; 1.06222893e+145 4.19293595e+144
NL: Newton 20 ; 3.90779737e+144 1.54252474e+144
NL: Newton 21 ;

In [7]:
assert_near_equal(p['circuit.n1.V'], 9.98744708, 1e-5)
assert_near_equal(p['circuit.n2.V'], 8.73215484, 1e-5)
assert_near_equal(p['circuit.R1.I'] + p['circuit.D1.I'], 0.09987447, 1e-6)

7.776244532022507e-09

```{important}
You actually *can* get this model to converge. But you have to set the options for `maxiter=400` and `rtol=1e-100`. (The `rtol` value needs to be very low to prevent premature termination.)
```

## Tweaking Newton Solver Settings to Get More Robust Convergence
---

The :ref:`NewtonSolver <nlnewton>` has a lot of features that allow you to modify its behavior to handle more challenging problems. We're going to look at two of the most important ones here:

1. :ref:`Line searches <feature_line_search>`
2. The *solve_subsystems* option

If we use both of these in combination, we can dramatically improve the robustness of the solver for this problem.
The *linesearch* attribute makes sure that the solver doesn't take too big of a step. The *solve_subsystems* option allows the `Resistor` and `Diode` components (the two `ExplicitComponents`) to help the convergence by updating their own output values given their inputs. When you use :ref:`NewtonSolver <nlnewton>` on models with a lot of `ExplicitComponents`, you may find that turning on *solve_subsystems* helps convergence, but you need to be careful about the :ref:`execution order <feature_set_order>` when you try this.

```{important}
For this case, we used the :ref:`ArmijoGoldsteinLS <feature_armijo_goldstein>`, which basically limits step sizes so that the residual always goes down. For many problems you might want to use :ref:`BoundsEnforceLS <feature_bounds_enforce>` instead, which only activates the line search to enforce upper and lower bounds on the outputs in the model.
```

In [ ]:
from openmdao.test_suite.scripts.circuit_analysis import Circuit

p = om.Problem()
model = p.model

model.add_subsystem('ground', om.IndepVarComp('V', 0., units='V'))
model.add_subsystem('source', om.IndepVarComp('I', 0.1, units='A'))
model.add_subsystem('circuit', Circuit())

model.connect('source.I', 'circuit.I_in')
model.connect('ground.V', 'circuit.Vg')

p.setup()

# you can change the NewtonSolver settings in circuit after setup is called
newton = p.model.circuit.nonlinear_solver
newton.options['iprint'] = 2
newton.options['maxiter'] = 10
newton.options['solve_subsystems'] = True
newton.linesearch = om.ArmijoGoldsteinLS()
newton.linesearch.options['maxiter'] = 10
newton.linesearch.options['iprint'] = 2

# set some initial guesses
p['circuit.n1.V'] = 10.
p['circuit.n2.V'] = 1e-3

p.run_model()

print(p['circuit.n1.V'])
print(p['circuit.n2.V'])
print(p['circuit.R1.I'])
print(p['circuit.R2.I'])
print(p['circuit.D1.I'])

# sanity check: should sum to .1 Amps
print(p['circuit.R1.I'] + p['circuit.D1.I'])

In [ ]:
assert_near_equal(p['circuit.n1.V'], 9.90804735, 1e-5)
assert_near_equal(p['circuit.n2.V'], 0.71278185, 1e-5)
assert_near_equal(p['circuit.R1.I'], 0.09908047, 1e-5)
assert_near_equal(p['circuit.R2.I'], 0.00091953, 1e-5)
assert_near_equal(p['circuit.D1.I'], 0.00091953, 1e-5)

# sanity check: should sum to .1 Amps
assert_near_equal(p['circuit.R1.I'] + p['circuit.D1.I'], .1, 1e-6)

```{important}
This tutorial used finite difference to approximate the partial derivatives for all the components. Check out :ref:`this example <circuit_analysis_examples>` if you want to see the same problem solved with analytic derivatives.
```